### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,accuracy_score,\
log_loss,confusion_matrix
import math
import warnings
warnings.filterwarnings('ignore')

### Reading Data

In [2]:
df=pd.read_csv("bank.xls")

In [3]:
df.drop(['ID'],axis=1,inplace=True)

### Downsampling Data

In [4]:
one_class=df[df['Personal Loan']==1]
zero_class=df[df['Personal Loan']==0]

In [5]:
zero_downsample = resample(zero_class,
             replace=True,
             n_samples=len(one_class),
             random_state=42)

In [6]:
data=pd.concat([zero_downsample,one_class])

In [7]:
X=data.drop(['Personal Loan'],axis=1).values
Y=data['Personal Loan'].values

### Splitting Data

In [8]:
X_train,X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [9]:
X_train.shape

(768, 12)

In [10]:
sc=StandardScaler()

In [11]:
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

### Particle Swarm Optimisation

In [12]:
import numpy as np
from sklearn.metrics import log_loss, accuracy_score


class ParticleSwarmOptimization:

    def __init__(self, X_train,y_train, num_particles=30, hidden_layer=10):
        self.X_train = X_train
        self.y_train = y_train
        self.num_particles = num_particles
        self.hidden_layer = hidden_layer
        self.input_layer = X_train.shape[1]
        self.output_layer = 1
        self.weights = self.input_layer*self.hidden_layer+self.hidden_layer+self.hidden_layer*self.output_layer+self.output_layer
        self.positions = np.random.rand(self.num_particles, self.weights)*2-1
        self.velocities = np.random.rand(self.num_particles, self.weights)*2-1
        self.personal_bests = self.positions.copy()
        self.w = 0.7
        self.c1 = self.c2 = 2
        self.best_weights=None

    def sigmoid(self, x):
        return 1/(1 + np.exp(-x))

    def tanh(self, x):
        return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))

    def relu(self, x):
        return np.maximum(0, x)

    def forward(self, inputs, weights):
        weight1 = np.reshape(weights[:self.hidden_layer*self.input_layer], (self.hidden_layer, self.input_layer))
        bias1 = np.reshape(weights[self.hidden_layer*self.input_layer:self.hidden_layer*self.input_layer+self.hidden_layer], (self.output_layer, self.hidden_layer))
        weight2 = np.reshape(weights[self.hidden_layer*self.input_layer+self.hidden_layer:-1], (self.output_layer, self.hidden_layer))
        bias2 = np.reshape(weights[-1], (self.output_layer, self.output_layer))

        A0 = inputs
        Z1 = inputs.dot(weight1.T)+bias1
        A1 = self.relu(Z1)
        Z2 = A1.dot(weight2.T)+bias2
        A2 = self.sigmoid(Z2)
        y_pred = np.round(A2)

        return y_pred

    def fitness_function(self, inputs, positions, y_train):
        fitness = []
        for position in positions:
            y_pred = self.forward(inputs, position)
            loss = log_loss(y_train, y_pred)
            fitness.append(loss)

        return fitness

    def find_Gbest(self, inputs, positions, y_train):
        gbest = 0
        gbest_index = -1

        for i in range(self.num_particles):
            temp = accuracy_score(y_train, self.forward(inputs, positions[i]))
            if temp > gbest:
                gbest = temp
                gbest_index = i

        return gbest_index

    def compute_velocities(self, positions, velocities, personal_bests):
        global_best = positions[self.find_Gbest(self.X_train, positions, self.y_train)]

        new_velocities = self.w*velocities + self.c1*np.random.rand()*(personal_bests-positions) \
                       + self.c2*np.random.rand()*(global_best-positions)

        return new_velocities

    def compute_new_positions(self, positions, velocities, personal_bests):
        new_positions = positions + self.compute_velocities(positions, velocities, personal_bests)
        new_personal_bests = personal_bests

        for i in range(len(positions)):
            old_fitness = accuracy_score(self.y_train, self.forward(self.X_train, positions[i]))
            new_fitness = accuracy_score(self.y_train, self.forward(self.X_train, new_positions[i]))

            if old_fitness < new_fitness:
                new_personal_bests[i] = new_positions[i]
          
        return new_positions, new_personal_bests
                
    def train(self, num_iterations):
        for i in range(num_iterations):
            fitness = self.fitness_function(self.X_train, self.positions, self.y_train)
            personal_best_fitness = self.fitness_function(self.X_train, self.personal_bests, self.y_train)

            for j in range(self.num_particles):
                if fitness[j] < personal_best_fitness[j]:
                    self.personal_bests[j] = self.positions[j]

            self.positions, self.personal_bests = self.compute_new_positions(self.positions, self.velocities, self.personal_bests)
            
            print(f"Iteration {i}\nGlobal Best : {accuracy_score(y_train,self.forward(self.X_train,self.positions[self.find_Gbest(self.X_train, self.positions, self.y_train)]))}")

        best_index = self.find_Gbest(self.X_train, self.positions, self.y_train)
        self.best_weights = self.positions[best_index]
    
    def predict(self,X_test):
        prediction=self.forward(X_test,self.best_weights)
        return prediction
    
    def reports(self,X_test,y_test):
        prediction=self.forward(X_test,self.best_weights)
        
        print(f"ACCURACY : {accuracy_score(y_test,prediction)}\n\n\
CLASSIFICATION REPORT :\n {classification_report(y_test,prediction)}")


In [13]:
pso=ParticleSwarmOptimization(X_train,y_train)

In [14]:
pso.train(30)

Iteration 0
Global Best : 0.7434895833333334
Iteration 1
Global Best : 0.77734375
Iteration 2
Global Best : 0.8177083333333334
Iteration 3
Global Best : 0.83203125
Iteration 4
Global Best : 0.8177083333333334
Iteration 5
Global Best : 0.81640625
Iteration 6
Global Best : 0.8385416666666666
Iteration 7
Global Best : 0.8541666666666666
Iteration 8
Global Best : 0.83984375
Iteration 9
Global Best : 0.828125
Iteration 10
Global Best : 0.8372395833333334
Iteration 11
Global Best : 0.8307291666666666
Iteration 12
Global Best : 0.8489583333333334
Iteration 13
Global Best : 0.8450520833333334
Iteration 14
Global Best : 0.8463541666666666
Iteration 15
Global Best : 0.8489583333333334
Iteration 16
Global Best : 0.8528645833333334
Iteration 17
Global Best : 0.85546875
Iteration 18
Global Best : 0.8645833333333334
Iteration 19
Global Best : 0.8580729166666666
Iteration 20
Global Best : 0.8619791666666666
Iteration 21
Global Best : 0.8541666666666666
Iteration 22
Global Best : 0.84765625
Iteration 

In [15]:
pso.reports(X_test,y_test)

ACCURACY : 0.8854166666666666

CLASSIFICATION REPORT :
               precision    recall  f1-score   support

           0       0.89      0.85      0.87        86
           1       0.88      0.92      0.90       106

    accuracy                           0.89       192
   macro avg       0.89      0.88      0.88       192
weighted avg       0.89      0.89      0.89       192

